<a href="https://www.kaggle.com/code/dianaddx/ferrari-n-v-stock-market-prediction-w-prophet?scriptVersionId=160172166" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="padding:20px; 
            color:black;
            margin:10px;
            font-size:200%;
            text-align:center;
            display:fill;
            border-radius:20px;
            border-width: 5px;
            border-style: solid;
            border-color:#eb0205;
            background-color:#eb0205;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">Exploring <i>Time Series Forecasting</i> with <b>Prophet</b><br> - Predicting <b>Ferrari</b> <i>Stock Prices</i> -            
 </div> 
 
<br>

<center>
<img src="https://cdn.worldvectorlogo.com/logos/ferrari-ges.svg" width=200>
</center>

<br>
<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:140%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    This project is focused on <b>time series forecasting</b> for the <b>stock price</b> of Ferrari N.V. (ticker symbol <i>RACE</i>) using the <b>Prophet algorithm</b>. Prophet, <u>created by Facebook</u>, is an open-source package for time series forecasting. It is specifically engineered to manage time series data with multiple seasonal patterns.
The notebook combines data retrieval, data preparation, modeling, forecasting and performance evaluation to provide insights into the stock price of Ferrari N.V. and assess the predictive accuracy of the Prophet algorithm. It is a practical application of time series forecasting in the context of financial data.          
 </div> 


---
<br>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#28990b;font-family:newtimeroman;color:black;font-size:160%;text-align:center;border-radius:30px 10px;">TABLE OF CONTENTS</p>   
    
    
1.  [Import Libraries](#1)
    
2.  [Data Loading](#2)
    
3.  [Model Building](#3)  

4.  [Forecasting](#4)
    
5.  [Performance Metrics](#5)
    
<br>
    
---

<a id="1"></a>
<p style="background-color:#28990b;font-family:newtimeroman;color:#black;font-size:150%;text-align:center;border-radius:30px 10px;">Import Libraries</p>



In [1]:
pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-linux_x86_64.whl size=290463 sha256=23480441af8b41d01e6be94de94abd4cdd2eba6ad3cdc4579b2cb1ea26fb855d
  Stored in directory: /root/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

import yfinance as yf
from prophet import Prophet

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
import plotly.graph_objects as go
from prophet.plot import plot_plotly, plot_components_plotly
from tabulate import tabulate

import datetime
from datetime import date, datetime, timedelta

from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

<a id="2"></a>
<p style="background-color:#28990b;font-family:newtimeroman;color:#black;font-size:150%;text-align:center;border-radius:30px 10px;">Data Loading</p>


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    Historical stock price data for Ferrari is obtained from <b>Yahoo Finance</b> using the <b>yfinance</b> library.         
 </div> 


In [3]:
import datetime
# Define the ticker symbol for Ferrari
ticker_symbol = 'RACE'

# Define the start and end dates for the data 
start_date = '2015-10-21'
end_date = datetime.date.today().strftime('%Y-%m-%d')

# Fetch data from Yahoo Finance
ferrari_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Create a DataFrame with just the date and closing price
ferrari_df = ferrari_data[['Adj Close']].reset_index()
ferrari_df = ferrari_df.rename(columns={'Date': 'ds', 'Adj Close': 'y'})

ferrari_df.head()
# ferrari_df.tail()


[*********************100%%**********************]  1 of 1 completed


,ds,y
0,2015-10-21,51.579483
1,2015-10-22,53.220646
2,2015-10-23,52.873657
3,2015-10-26,51.598240
4,2015-10-27,50.500999


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    The retrieved data is transformed into a pandas DataFrame, with columns <i>ds</i> for dates and <i>y</i> for the adjusted closing prices. This format is suitable for use with the Prophet algorithm. The code is updated to automatically set the end date to the current date, ensuring that the most recent data is used for forecasting.         
 </div> 




<a id="3"></a>
<p style="background-color:#28990b;font-family:newtimeroman;color:#black;font-size:150%;text-align:center;border-radius:30px 10px;">Model Building</p>


In [4]:
#  instantiating a new Prophet object
m = Prophet()

#  fit the model
m.fit(ferrari_df)

20:12:03 - cmdstanpy - INFO - Chain [1] start processing
20:12:05 - cmdstanpy - INFO - Chain [1] done processing


<a id="4"></a>
<p style="background-color:#28990b;font-family:newtimeroman;color:#black;font-size:150%;text-align:center;border-radius:30px 10px;">Forecasting</p>

In [5]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2436,2025-01-17
2437,2025-01-18
2438,2025-01-19
2439,2025-01-20
2440,2025-01-21


In [6]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2436,2025-01-17,461.447494,383.014070,542.128293
2437,2025-01-18,465.092697,386.206651,543.896813
2438,2025-01-19,465.319358,388.253133,543.554415
2439,2025-01-20,462.436250,387.115321,541.687935
2440,2025-01-21,462.488891,385.651216,541.938742


In [7]:
fig = plot_plotly(m, forecast)
# Update the color attributes to use red
fig.update_traces(marker=dict(color='red'), line=dict(color='black'))

In [8]:
plot_components_plotly(m, forecast).update_traces(line=dict(color='black'))

<a id="5"></a>
<p style="background-color:#28990b;font-family:newtimeroman;color:#black;font-size:150%;text-align:center;border-radius:30px 10px;">Performance Metrics</p>


# Mean Absolute Error (MAE)

In [9]:
mae = mean_absolute_error(ferrari_df['y'], forecast['yhat'][:len(ferrari_df)])
print(f"Mean Absolute Error (MAE): {mae:.3f}")

Mean Absolute Error (MAE): 7.365


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    The Mean Absolute Error measures the average absolute difference between the actual values and the model's predictions. In this case, the MAE of <b>8.67</b> suggests that, on average, the model's predictions deviate from the actual values by approximately 8.67 units. Lower MAE values are desirable, indicating a more accurate model.      
 </div> 



# Mean Squared Error (MSE)

In [10]:
mse = mean_squared_error(ferrari_df['y'], forecast['yhat'][:len(ferrari_df)])
print(f"Mean Squared Error (MSE): {mse:.3f}")



Mean Squared Error (MSE): 92.422


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    The Mean Squared Error measures the average squared difference between the actual and predicted values. With an MSE of <b>137.746</b>, the model's errors are squared before averaging, making it sensitive to larger errors. The lower the MSE, the better, and a value of 0 would represent a perfect match.     
 </div> 

# Root Mean Squared Error (RMSE)

In [11]:
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")

Root Mean Squared Error (RMSE): 9.614


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#fffb05;
            background-color:#fffb05;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    The Root Mean Squared Error is the square root of the MSE, expressed in the same units as the data. An RMSE of <b>11.737</b> suggests that, on average, the model's predictions deviate from the actual values by approximately 11.737 units. Like MAE and MSE, lower RMSE values indicate a more accurate model.    
 </div> 


In [12]:
# Metrics Table
table_data = [
    ["Metric", "Value"],
    ["Mean Absolute Error (MAE)", f"{mae:.3f}"],
    ["Mean Squared Error (MSE)", f"{mse:.3f}"],
    ["Root Mean Squared Error (RMSE)", f"{rmse:.3f}"]
]

table = tabulate(table_data, headers="firstrow", tablefmt="fancy_grid")

print(table)


╒════════════════════════════════╤═════════╕
│ Metric                         │   Value │
╞════════════════════════════════╪═════════╡
│ Mean Absolute Error (MAE)      │   7.365 │
├────────────────────────────────┼─────────┤
│ Mean Squared Error (MSE)       │  92.422 │
├────────────────────────────────┼─────────┤
│ Root Mean Squared Error (RMSE) │   9.614 │
╘════════════════════════════════╧═════════╛


<div style="padding:10px; 
            color:black;
            margin:10px;
            font-size:130%;
            text-align:left;
            display:fill;
            border-radius:30px;
            border-width: 5px;
            border-style: solid;
            border-color:#eb0205;
            background-color:#eb0205;
            font-family:newtimeroman;
            overflow:hidden;
            font-weight:500">
    <p style="margin-bottom: 10px; font-weight: bold;">Conclusion:</p>
        <ul> 
            
   * in this study, we applied the Prophet algorithm to forecast Ferrari's stock prices based on historical data sourced from Yahoo Finance;
           
   * our assessment of the model's performance reveals meaningful insights;

   * the model demonstrates predictive capability, although it yields a Mean Absolute Error (MAE) of 8.67, Mean Squared Error (MSE) of 137.746, and Root Mean Squared Error (RMSE) of 11.737; these metrics underscore the model's accuracy and forecasting precision;
            
   * nonetheless, the inherent volatility and unpredictability of financial markets present challenges for forecasting; the model serves as a foundation for refinement and customization, offering valuable insights into stock price trends;
            
   * ongoing monitoring, model adaptation, and the inclusion of domain expertise and external factors are pivotal for enhancing predictive accuracy in this dynamic environment; as we continually improve our data and techniques, the future holds promise for more precise and insightful stock price predictions;
    </ul>    
 </div> 